Installing scanpy and defining file paths for two anndata objects

In [1]:
file_path_brain = 'Mouse_brain_cell_bin.h5ad'
file_path_embryo = 'E9.5_E1S1.MOSTA.h5ad'

In [2]:
!pip install scanpy

Importing necessary packages and reading anndata objects with scanpy

In [3]:

import numpy as np
import scanpy as sc

In [4]:
anndata_brain=sc.read(file_path_brain)
anndata_embryo=sc.read(file_path_embryo)

Only considering the two last: ['.MOSTA', '.h5ad'].
Only considering the two last: ['.MOSTA', '.h5ad'].


There are two steps in preprocessing section:

Filtering - removing genes which are pressent in low number of cells (less than 10)

Normalization and log transformation - ensuring that cells participate equally in analysis and reducing range of expression values (stabilizes variance). This improves performance of analysis

In [5]:
sc.pp.filter_genes(anndata_embryo, min_cells=10)
sc.pp.filter_genes(anndata_brain, min_cells=10)
sc.pp.normalize_total(anndata_embryo, inplace=True)
sc.pp.log1p(anndata_embryo)
sc.pp.normalize_total(anndata_brain, inplace=True)
sc.pp.log1p(anndata_brain)

In [6]:
print(len(anndata_embryo.obs_names))
print(len(anndata_embryo.var_names))
print(len(anndata_brain.obs_names))
print(len(anndata_brain.var_names))

5913
20055
50140
20379


Python implementation of algorithm for detecting SVG using Moran's I statistic.
First, distance matrix is calculated for spatial coordinates of anndata object, and based on that matrix, weight matrix is calculated as inversed distances. After we have weight matrix, we can calculate Moran's I score for each gene using parallel computation. For each gene morans_i function is called with two parameters: gene_expression and weight matrix.
In morans_i function we need to calculate mean_expression and expression_diff so that we can use those values in Moran's I formula. After implementing formula, Moran's I score is returned and based on that score and threshold, it is decided whether gene is spatially variable. Finally, list of spatially variable genes is returned.

In [7]:
from scipy.spatial import distance_matrix
from joblib import Parallel, delayed
import numba
import pandas as pd

@numba.jit(nopython=True)
def morans_i(gene_expression,weight):
    mean_expression=np.mean(gene_expression)
    expression_diff=gene_expression-mean_expression
    numerator=np.sum(weight*np.outer(expression_diff,expression_diff))
    denominator=np.sum(expression_diff**2)
    morans=(len(gene_expression)/np.sum(weight))*(numerator/denominator)
    return morans

def compute_svgs(anndata,threshold):
    spatial=anndata.obsm['spatial']
    genes=anndata.var_names
    gene_expressions=anndata.X
    dist_matrix=distance_matrix(spatial,spatial)
    np.fill_diagonal(dist_matrix,np.inf)
    weight=1/dist_matrix
    results=Parallel(n_jobs=36)(delayed(morans_i)(gene_expressions[:,gene].toarray().flatten(),weight) for gene in range(gene_expressions.shape[1]))
    morans_i_series = pd.Series(results, index=genes)
    print(morans_i_series.describe())
    print(morans_i_series.head(10)) 
    svg_genes = [genes[i] for i, morans_i in enumerate(results) if morans_i > threshold]
    return svg_genes

Calculating SVG for embryo

In [31]:
threshold=0.01

svg_embryo=compute_svgs(anndata_embryo,threshold)

print(len(svg_embryo))

count    20055.000000
mean         0.004387
std          0.011410
min         -0.000896
25%          0.000151
50%          0.001178
75%          0.003691
max          0.206134
dtype: float64
gene_short_name
1700007G11Rik    0.004909
1700123O20Rik    0.002575
1810030O07Rik    0.001203
2010107E04Rik    0.011950
2210016F16Rik    0.000469
2600014E21Rik    0.004014
2610001J05Rik    0.002519
2810429I04Rik    0.002157
3830406C13Rik    0.000383
4930481A15Rik    0.011499
dtype: float64
1998


Saving svg_embryo to txt file

In [24]:
def save_genes_to_txt(genes, filename):
    with open(filename, 'w') as file:
        for gene in genes:
            file.write(gene + '\n')

In [32]:
embryo_filename = 'svg_genes_embryo_moranI.txt'

save_genes_to_txt(svg_embryo, embryo_filename)

Basically same algorithm used for calculating SVG for embryo, but with optimizations in calculating weight matrix and passing weight matrix to morans_i as sparse matrix. Optimizations are done because of large spatial matrix in anndata_brain object

In [18]:
from scipy.spatial import cKDTree
from scipy.sparse import csr_matrix

def morans_i_sparse(gene_expression,weight_data,weight_indices,weight_ptr):
    mean_expression=np.mean(gene_expression)
    expression_diff=gene_expression-mean_expression
    numerator=0
    for i in range(len(gene_expression)):
        for j in range(weight_ptr[i],weight_ptr[i+1]):
            k=weight_indices[j]
            numerator+=weight_data[j]*expression_diff[i]*expression_diff[k]
    denominator=np.sum(expression_diff**2)
    morans=(len(gene_expression)/np.sum(weight_data))*(numerator/denominator)
    return morans
    

def compute_weight(spatial,k=5):
    tree=cKDTree(spatial)
    distances,indices=tree.query(spatial,k=k+1)
    distances=distances[:,1:]
    indices=indices[:,1:]
    weights=1/distances
    rows=np.repeat(np.arange(spatial.shape[0]),k)
    cols=indices.flatten()
    data=weights.flatten()
    weight_mat=csr_matrix((data,(rows,cols)),shape=(spatial.shape[0],spatial.shape[0]))
    return weight_mat

def compute_svgs_batch(anndata,threshold,k=5):
    spatial=anndata.obsm['spatial']
    genes=anndata.var_names
    gene_expressions=anndata.X
    weight_mat=compute_weight(spatial,k=k)
    weight_data=weight_mat.data
    weight_indices=weight_mat.indices
    weight_ptr=weight_mat.indptr

    def process_gene(gene_id):
        gene_expression=gene_expressions[:,gene_id].toarray().flatten()
        return morans_i_sparse(gene_expression,weight_data,weight_indices,weight_ptr)

    results=Parallel(n_jobs=48)(delayed(process_gene)(gene) for gene in range(gene_expressions.shape[1]))
    morans_i_series = pd.Series(results, index=genes)
    print(morans_i_series.describe())
    print(morans_i_series.head(10)) 
    svg_genes = [genes[i] for i, morans_i in enumerate(results) if morans_i > threshold]
    return svg_genes

Calculating SVG for brain

In [22]:
threshold=0.02

svg_brain=compute_svgs_batch(anndata_brain,threshold)

print(len(svg_brain))

count    20379.000000
mean         0.007564
std          0.021108
min         -0.008232
25%         -0.000432
50%          0.001960
75%          0.007391
max          0.509993
dtype: float64
0610005C13Rik   -0.000759
0610009B22Rik    0.002740
0610009O20Rik   -0.002803
0610010F05Rik    0.005186
0610010K14Rik    0.004142
0610012G03Rik    0.008426
0610025J13Rik   -0.000152
0610030E20Rik    0.002169
0610037L13Rik    0.004722
0610038B21Rik    0.001794
dtype: float64
1954


Saving svg_brain to txt file

In [25]:
brain_filename = 'svg_genes_brain_moranI.txt'

save_genes_to_txt(svg_brain, brain_filename)